# Counting Crowds with DL
## Proof Of Concept
The notebook will implement [Dense Scale Networks](https://arxiv.org/pdf/1906.09707.pdf) for the purpose of counting crowds for images.
The dataset will be the same as the one, detailed in the paper - the ShangaiTech.

## 1. Imports and loading the data.

In [2]:
import os
import pandas as pd
import numpy as np
import keras
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Dense, Dropout, Flatten
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.metrics import mse, mae

In [20]:
import scipy.io

gt_file = scipy.io.loadmat('training_dataset/ShanghaiTech/part_A/train_data/ground-truth/GT_IMG_1.mat')

In [43]:
gt_file['image_info']

array([[array([[(array([[ 29.6225116 , 472.92022152],
       [ 54.35533603, 454.96602305],
       [ 51.79045053, 460.46220626],
       ...,
       [597.89732076, 688.27900015],
       [965.77518336, 638.44693908],
       [166.9965574 , 628.1873971 ]]), array([[1546]], dtype=uint16))]],
      dtype=[('location', 'O'), ('number', 'O')])]], dtype=object)